<a href="https://colab.research.google.com/github/ZLRock/ColabLink/blob/main/Tortillasquad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar la librería para el manejo de matrices y adjuntando el archivo .csv

In [18]:
import pandas as pd

In [19]:
from google.colab import files
uploaded = files.upload()

In [20]:
tortillacsv = pd.read_csv('tortilla_prices.csv')

Dividir los datos para hacer entrenamiento con el 90% y 10% para hacer validaciones

In [23]:
df = tortillacsv.sample(frac=1, random_state=42).reset_index(drop=True)

train_pct = 0.9
train_size = int(len(df) * train_pct)

traintortilla = df[:train_size]
testtortilla = df[train_size:]

In [24]:
traintortilla.describe()

,Year,Month,Day,Price per kilogram
count,260231.000000,260231.000000,260231.000000,254433.000000
mean,2015.398346,6.383917,15.932602,12.330057
std,5.098110,3.404789,8.745221,4.615097
min,2007.000000,1.000000,1.000000,0.000000
25%,2011.000000,3.000000,8.000000,9.640000
50%,2015.000000,6.000000,16.000000,11.300000
75%,2020.000000,9.000000,24.000000,14.230000
max,2024.000000,12.000000,31.000000,31.000000


Quitando los valores NaN

In [25]:
tortillaclean = traintortilla.dropna(axis = 0)
tortillaclean

,State,City,Year,Month,Day,Store type,Price per kilogram
0,Querétaro,Querétaro,2016,12,5,Mom and Pop Store,14.00
1,Durango,Durango,2013,11,22,Big Retail Store,9.50
2,Sinaloa,Culiacán,2007,7,16,Mom and Pop Store,8.50
3,Tlaxcala,Tlaxcala,2015,4,22,Mom and Pop Store,10.00
4,Sonora,Hermosillo,2017,10,27,Mom and Pop Store,19.83
...,...,...,...,...,...,...,...
260226,D.F.,D.F.,2017,11,3,Mom and Pop Store,12.83
260227,Aguascalientes,Aguascalientes,2018,3,7,Big Retail Store,9.73
260228,Puebla,ZM Puebla,2023,3,31,Big Retail Store,12.43
260229,Jalisco,Guadalajara,2010,2,15,Big Retail Store,6.10


Definiendo las columnas de entrenamiento y la columna de resultados

In [26]:
y = tortillaclean['Price per kilogram']
X = tortillaclean[['Year', 'Month', 'Day']]

Empieza el modelo de machine learning del tipo Tree Regressor

In [54]:
from sklearn.tree import DecisionTreeRegressor

tortilla_model = DecisionTreeRegressor()
tortilla_model.fit(X, y)

DecisionTreeRegressor()

Primera prueba utilizando diez valores para hacer una prueba y comparándolo con los datos reales

In [28]:
print("Predicción de el precio según la fecha")
print(X.head(10))
print("El precio real fue:")
print(y.head(10))
print("El precio predecido es:")
print(tortilla_model.predict(X.head(10)))

Predicción de el precio según la fecha
   Year  Month  Day
0  2016     12    5
1  2013     11   22
2  2007      7   16
3  2015      4   22
4  2017     10   27
5  2024      8   28
6  2008      5   28
7  2009      8   24
8  2023      7   31
9  2010      1   20
El precio real fue:
0    14.00
1     9.50
2     8.50
3    10.00
4    19.83
5    26.00
6     5.87
7     6.13
8    14.67
9     6.77
Name: Price per kilogram, dtype: float64
El precio predecido es:
[11.98949495 11.52792079  7.39989247 11.65660194 12.62729167 19.5501
  7.63923913  8.28851064 19.34284211  8.47742268]


Sacando la media absoluta de error

In [29]:
from sklearn.metrics import mean_absolute_error

predicted_tortilla_prices = tortilla_model.predict(X)
mean_absolute_error(y, predicted_tortilla_prices)


2.5198238615535353

Validando datos con el 10% que se dejó para pruebas

In [41]:
train_X = tortillaclean[['Year', 'Month', 'Day']]
train_y = tortillaclean['Price per kilogram']

clean_tortillatest= testtortilla.dropna(axis = 0)

val_X = clean_tortillatest[['Year', 'Month', 'Day']]
val_y = clean_tortillatest['Price per kilogram']

from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

tortilla_model = DecisionTreeRegressor()
tortilla_model.fit(train_X, train_y)

val_predictions = tortilla_model.predict(val_X)

mae = mean_absolute_error(val_y, val_predictions)
print(mae)

2.5522111333726127


Utilizando diferentes cantidades de hojas para verificar overfeeding y underfeeding

In [46]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [48]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %f" %(max_leaf_nodes, float(my_mae)))

Max leaf nodes: 5  		 Mean Absolute Error:  2.544872
Max leaf nodes: 50  		 Mean Absolute Error:  2.530130
Max leaf nodes: 500  		 Mean Absolute Error:  2.541066
Max leaf nodes: 5000  		 Mean Absolute Error:  2.552211


Utilizando otro modelo para verificar el error medio

In [49]:
from sklearn.ensemble import RandomForestRegressor

tortillaforest = RandomForestRegressor(random_state=1)
tortillaforest.fit(train_X, train_y)
tortilla_preds = tortillaforest.predict(val_X)
print(mean_absolute_error(val_y, tortilla_preds))

2.5522613242788217


Tratando de predecir el precio de hoy

In [58]:
precio_hoy = [[2024, 1, 22]]
print(tortilla_model.predict(precio_hoy))

[19.56102041]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
